In [1]:

seed_val = 1304

import os
####*IMPORANT*: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED']=str(seed_val)

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers 
import random
import pandas as pd
import numpy as np

tf.random.set_seed(seed_val)
np.random.seed(seed_val)
random.seed(seed_val)

from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers, optimizers, regularizers
from sklearn import ensemble

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Google_Stock_Price_Train.csv', thousands=',')
df_test = pd.read_csv('Google_Stock_Price_Test.csv', thousands=',')
df.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,7380500
1,1/4/2012,331.27,333.87,329.08,666.45,5749400
2,1/5/2012,329.83,330.75,326.89,657.21,6590300
3,1/6/2012,328.34,328.77,323.68,648.24,5405900
4,1/9/2012,322.04,322.29,309.46,620.76,11688800


In [3]:
df['Mid'] = (df['High'] + df['Low']) / 2
df_test['Mid'] = (df_test['High'] + df_test['Low']) / 2
df.head()

,Date,Open,High,Low,Close,Volume,Mid
0,1/3/2012,325.25,332.83,324.97,663.59,7380500,328.900
1,1/4/2012,331.27,333.87,329.08,666.45,5749400,331.475
2,1/5/2012,329.83,330.75,326.89,657.21,6590300,328.820
3,1/6/2012,328.34,328.77,323.68,648.24,5405900,326.225
4,1/9/2012,322.04,322.29,309.46,620.76,11688800,315.875


In [4]:
df['Up Down'] = np.where(df['Close'].shift(1) > df['Close'], 0, 1)
df_test['Up Down'] = np.where(df_test['Close'].shift(1) > df_test['Close'], 0, 1)

df['VolumeBand'] = pd.cut(df['Volume'], 4)
df[['VolumeBand', 'Up Down']].groupby( ['VolumeBand'], as_index=False).mean().sort_values(by='VolumeBand', ascending=True)

,VolumeBand,Up Down
0,"(-17070.0, 6250400.0]",0.508906
1,"(6250400.0, 12492900.0]",0.514286
2,"(12492900.0, 18735400.0]",0.600000
3,"(18735400.0, 24977900.0]",0.250000


In [5]:
df['High Volume'] = np.where(df['Volume'] > 18735400.0, 1, 0)
df = df.drop('VolumeBand', axis=1)
df_test['High Volume'] = np.where(df_test['Volume'] > 18735400.0, 1, 0)
df.head()

,Date,Open,High,Low,Close,Volume,Mid,Up Down,High Volume
0,1/3/2012,325.25,332.83,324.97,663.59,7380500,328.900,1,0
1,1/4/2012,331.27,333.87,329.08,666.45,5749400,331.475,1,0
2,1/5/2012,329.83,330.75,326.89,657.21,6590300,328.820,0,0
3,1/6/2012,328.34,328.77,323.68,648.24,5405900,326.225,0,0
4,1/9/2012,322.04,322.29,309.46,620.76,11688800,315.875,0,0


In [6]:
feature = ['Open', 'Close', 'Mid', 'High Volume']

x = df[feature]
y = df['Up Down']

x_test = df_test[feature]
y_test = df_test['Up Down']

### Logistic Regression

In [7]:
lr = LogisticRegression(random_state=1200, solver='lbfgs', multi_class='multinomial').fit(x, y)
lr.predict(x_test)
lr.predict_proba(x_test)
lr_acc = lr.score(x_test,y_test)
print(lr_acc)

0.75


### Neural Networks

In [8]:
nn = models.Sequential()
nn.add(layers.Dense(12, input_shape=(4,), activation='relu'))
nn.add(layers.Dense(12, activation='relu'))
nn.add(layers.Dense(12, activation='relu'))
nn.add(layers.Dense(1, activation='sigmoid'))


from keras.optimizers import SGD
opt = SGD(lr=0.0001)
nn.compile( optimizer=opt, loss='binary_crossentropy', metrics=['accuracy']) 
nn.fit(x, y, epochs=20, batch_size=3)

nn.predict(x_test)
loss, nn_acc = nn.evaluate(x_test, y_test)
print(nn_acc)

Epoch 1/20
1258/1258 [==============================] - 1s 408us/step - loss: 1.6667 - accuracy: 0.4905
Epoch 2/20
1258/1258 [==============================] - 0s 314us/step - loss: 0.7872 - accuracy: 0.5215
Epoch 3/20
1258/1258 [==============================] - 0s 282us/step - loss: 0.7355 - accuracy: 0.5111
Epoch 4/20
1258/1258 [==============================] - 0s 288us/step - loss: 0.7330 - accuracy: 0.5056
Epoch 5/20
1258/1258 [==============================] - 0s 292us/step - loss: 0.7242 - accuracy: 0.5151
Epoch 6/20
1258/1258 [==============================] - 0s 293us/step - loss: 0.7295 - accuracy: 0.4825
Epoch 7/20
1258/1258 [==============================] - 0s 268us/step - loss: 0.7189 - accuracy: 0.5056
Epoch 8/20
1258/1258 [==============================] - 0s 281us/step - loss: 0.7071 - accuracy: 0.5127
Epoch 9/20
1258/1258 [==============================] - 0s 276us/step - loss: 0.7091 - accuracy: 0.5207
Epoch 10/20
1258/1258 [==============================] - 0s 268u

### Adaboost

In [9]:
boost = ensemble.AdaBoostClassifier(
    base_estimator=lr, n_estimators=100, learning_rate=2.0, random_state=2000
).fit(x, y)
boost.predict(x_test)
boost.predict_proba(x_test)
boost_acc = boost.score(x_test,y_test)
print(boost_acc)

0.65
